# Step 1: Producing data

In [ ]:
import json  # To convert dictionaries to JSON

class CsvConverter:
    """This class converts CSV data to JSON."""

    def __init__(self, keys_str):
        # The __init__ method runs when the class is instantiated.
        # Here we're setting the keys of our CSV data by splitting
        # the first line of the CSV on commas.
        self.keys = keys_str.split(',')
        
    def csv_to_json(self, csv_lines):
        # This method takes a list of CSV lines (strings)
        # and converts them to JSON.
        json_data = []  # We'll store our JSON data here
        
        for line in csv_lines:  # We loop over the CSV lines
            values = line.split(',')  # We split each line into values
            
            # We check if the number of keys matches the number of values
            # If not, we print a warning and skip this line
            if len(self.keys) != len(values):
                print("Warning: numbers of keys and values don't match in line:", line)
                continue
            
            # We create dictionary from keys and values using the zip function
            data_dict = dict(zip(self.keys, values))
            
            # We convert dictionary to a JSON string and add it to our list
            json_data.append(json.dumps(data_dict))
            
        return json_data  # We return the JSON data



In [ ]:
import linecache

class Reader:
    """This class reads a CSV file and returns JSON data."""

    def __init__(self, file_location, stride=5):
        self.file_location = file_location
        self.stride = stride # Number of lines to read at a time
        self.current_line = 1  # Starting from the first line

        # Instantiate CsvConverter
        # We need to read the first line of the CSV to get the keys
        keys_str = linecache.getline(
            self.file_location, self.current_line).rstrip('\n')
        self.converter = CsvConverter(keys_str)

    def get_lines(self):
        # Get the lines from the file
        start = self.current_line + 1
        # We add 1 to the current line because
        # we don't want to read the first line again
        end = start + self.stride

        lines = []
        for i in range(start, end):
            # We read the line and remove the "\n"
            line = linecache.getline(self.file_location, i).rstrip('\n')
            if line:
                lines.append(line)
            else:
                break

        self.current_line += len(lines)

        # Convert lines to JSON
        json_data = self.converter.csv_to_json(lines)
        return json_data


In [ ]:
reader = Reader('dSST.csv')
print(reader.get_lines())
print(reader.get_lines())
print(reader.get_lines())


___________________

___________

# Step 2: Consuming the data

In [ ]:
import pandas as pd
import json

class AverageYear:
    '''This class calculates the average temperature anomaly per year.'''
    def __init__(self, reader):
        self.reader = reader 
        self.dataframe = pd.DataFrame()

    def calculate_and_visualize(self):
        '''This method calculates the average temperature anomaly per year.'''

        while True: # Loop until there are no more lines
            lines = self.reader.get_lines()
            if not lines:
                break
            
            # Convert JSON lines to dictionary and create DataFrame
            data = [json.loads(line) for line in lines]
            df = pd.DataFrame(data)
            
            # Assuming the first column is 'Year', set it as the index
            df.set_index('Year', inplace=True)
            
            # Assuming the next 12 columns are 'Month', calculate their mean
            df['Average'] = df.iloc[:, 0:12].mean(axis=1)
            
            # Append to the main DataFrame
            self.dataframe = self.dataframe.append(df)
            
        # Display the average temperature anomaly per year
        print(self.dataframe['Average'])


class AverageMonth:
    
    def __init__(self, reader):
        self.reader = reader
        self.dataframe = pd.DataFrame()

    def calculate_and_visualize(self):
        while True:
            lines = self.reader.get_lines()
            if not lines:
                break
            
            # Convert JSON lines to dictionary and create DataFrame
            data = [json.loads(line) for line in lines]
            df = pd.DataFrame(data)
            print(df.head())
            # Assuming the first column is 'Year', set it as the index
            df.set_index('Year', inplace=True)
            
            # Assuming the next 12 columns are 'Month', transpose the DataFrame
            df = df.iloc[:, 0:12].transpose()
            
            # Calculate the average temperature anomaly per month
            df['Average'] = df.mean(axis=1)
            
            # Append to the main DataFrame
            self.dataframe = self.dataframe.append(df)
        
        # Display the average temperature anomaly per month
        print(self.dataframe['Average'])


In [ ]:
# Pass the dataset to the Reader class
reader = Reader('dSST.csv')

# Pass the reader to the AverageYear class
average_year = AverageYear(reader)

# Call the calculate_and_visualize method
average_year.calculate_and_visualize()

reader = Reader('dSST.csv')

average_month = AverageMonth(reader)
average_month.calculate_and_visualize()
